# Анализ данных сервиса о книгах

В распоряжении база данных конкурирующего сервиса. В ней содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги

### Цель

- провести исследовательский анализ данных сервиса-конкурента, благодаря которому станет возможно сформулировать ценностное предложение для нового продукта

### План

1. Подключение к базе данных и изучение содержащихся в ней таблиц
  - изучение таблиц базы данных
    - books
    - authors
    - publishers
    - ratings
    - reviews
  

2. Анализ данных
  - количество книг, выпущенных после 1 января 2000 года
    - вывод
  - количество пользовательских обзоров и средняя оценка для каждой книги
    - вывод
  - издательство, которое издало наибольшее число книг толще 50 страниц  
    - вывод
  - автор с самой высокой средней оценкой книг  
    - вывод
  - среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок  
    - вывод
    
    
3. Общие выводы

## 1. Подключение к базе данных и изучение содержащихся в ней таблиц

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# формируем опции подключения
# db_config = {'user': '', # имя пользователя
#              'pwd': '',  # пароль
#              'host': '', # хост
#              'port': '', # порт подключения
#              'db': ''}   # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def SQL_request_result(query):
    """ Функция, выводящая результат SQL-запроса
    
    принимает на вход SQL-запрос
    """
    
    return pd.io.sql.read_sql(query, con = engine)
    

### Изучение таблиц базы данных

Выведем первые строки таблиц

### books

In [3]:
# формируем запрос
query = """SELECT *
           FROM books
           LIMIT 1;
        """

# выводим результат
SQL_request_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


#### Описание данных

Содержит данные о книгах:

- `book_id` — идентификатор книги
- `author_id` — идентификатор автора
- `title` — название книги
- `num_pages` — количество страниц
- `publication_date` — дата публикации книги
- `publisher_id` — идентификатор издателя

### authors

In [4]:
# формируем запрос
query = """SELECT *
           FROM authors
           LIMIT 1;
        """

# выводим результат
SQL_request_result(query)

,author_id,author
0,1,A.S. Byatt


#### Описание данных
Содержит данные об авторах:

- `author_id` — идентификатор автора
- `author` — имя автора

### publishers

In [5]:
# формируем запрос
query = """SELECT *
           FROM publishers
           LIMIT 1;
        """

# выводим результат
SQL_request_result(query)

,publisher_id,publisher
0,1,Ace


#### Описание данных

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства
- `publisher` — название издательства

### ratings

In [6]:
# формируем запрос
query = """SELECT *
           FROM ratings
           LIMIT 1;
        """

# выводим результат
SQL_request_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


#### Описание данных

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки
- `book_id` — идентификатор книги
- `username` — имя пользователя, оставившего оценку
- `rating` — оценка книги

### reviews

In [7]:
# формируем запрос
query = """SELECT *
           FROM reviews
           LIMIT 1;
        """

# выводим результат
SQL_request_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


#### Описание данных

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора
- `book_id` — идентификатор книги
- `username` — имя пользователя, написавшего обзор
- `text` — текст обзора

## 2. Анализ данных

## Количество книг, выпущенных после 1 января 2000 года

In [8]:
# формируем запрос
query = """SELECT COUNT(book_id) AS number_of_books_after_date
           FROM books
           WHERE publication_date > '2000-01-01';
        """

In [9]:
# выводим результат
SQL_request_result(query)

,number_of_books_after_date
0,819


#### Вывод

Количество книг, выпущенных после 1 января 2000 года - 819. Это очень маленькое число книг, ориентироваться на него при запуске не стоит. Необходимо существенно увелисить данное число, чтобы оттянуть клиентов у конкурентов современными бестселлерами

## Количество пользовательских обзоров и средняя оценка для каждой книги

In [10]:
# формируем запрос
query = """SELECT books.title AS title,
                  SUBQ1.number_of_reviews,
                  ROUND(SUBQ2.average_rating, 2) AS average_rating
           FROM books
           INNER JOIN 
                (SELECT book_id,
                        COUNT(review_id) as number_of_reviews
                 FROM reviews
                 GROUP BY book_id) AS SUBQ1
                                         ON SUBQ1.book_id = books.book_id
           INNER JOIN 
                (SELECT book_id,
                        AVG(rating) as average_rating
                 FROM ratings
                 GROUP BY book_id) AS SUBQ2
                                         ON SUBQ2.book_id = books.book_id
           GROUP BY title,
                    SUBQ1.number_of_reviews,
                    average_rating
           ORDER BY SUBQ1.number_of_reviews DESC
           LIMIT 10;
        """

Ограничим отображение топ-10 книгами по числу пользовательских обзоров

In [11]:
# выводим результат
SQL_request_result(query)

,title,number_of_reviews,average_rating
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,The Da Vinci Code (Robert Langdon #2),6,3.83
3,The Lightning Thief (Percy Jackson and the Oly...,6,4.08
4,Outlander (Outlander #1),6,4.13
5,Eat Pray Love,6,3.40
6,Water for Elephants,6,3.98
7,The Hobbit or There and Back Again,6,4.13
8,The Book Thief,6,4.26
9,The Alchemist,6,3.79


#### Вывод

Наибольшее количество отзывов не влечет за собой высокие оценки для книг. По отзывам лидируют "Сумерки". В топ-10 книг по количеству отзывов преобладает фантастика. 

Скорее всего людям на карантине понравится отправляться в новые миры. Поэтому на фантастике при запуске не следует экономить. Однако, карантин - время развиваться, поэтому стоит рассмотреть добавление подборки книг по саморазвитию

## Издательство, которое издало наибольшее число книг толще 50 страниц

Принимаем, что книги менее 50 страниц - брошюры. Таким образом исключаем их из анализа

In [12]:
# формируем запрос
query = """SELECT publishers.publisher AS publisher_name,
                  COUNT (SUBQ.book_id) AS number_of_books
           FROM (
               SELECT book_id,
                      publisher_id
               FROM books
               WHERE num_pages > 50) AS SUBQ
           INNER JOIN publishers ON publishers.publisher_id = SUBQ.publisher_id
           GROUP BY publisher_name
           ORDER BY number_of_books DESC
           LIMIT 1;
        """

Выводим на экран издательство, выпустившее наибольшее число книг толще 50 страниц и количество выпущенных им книг

In [13]:
# выводим результат
SQL_request_result(query)

,publisher_name,number_of_books
0,Penguin Books,42


#### Вывод

В базе данных самое часто встречающееся издательство - "Penguin Books"

## Автор с самой высокой средней оценкой книг

Учитываются только книги с 50 и более пользовательскими оценками

In [14]:
# формируем запрос
query = """SELECT authors.author AS author_name,
                  ROUND(AVG(SUBQ.average_rating), 2) AS avg_rate    
           FROM books
           LEFT JOIN (
                   SELECT book_id,
                          COUNT(rating) AS cnt,
                          AVG(rating) AS average_rating
                   FROM ratings
                   GROUP BY book_id
                   ) AS SUBQ
                ON SUBQ.book_id = books.book_id
           LEFT JOIN authors ON authors.author_id = books.author_id
           WHERE SUBQ.cnt >= 50
           GROUP BY author_name
           ORDER BY avg_rate DESC
           LIMIT 1;   
        """

In [15]:
# выводим результат
SQL_request_result(query)

,author_name,avg_rate
0,J.K. Rowling/Mary GrandPré,4.28


#### Вывод

Наибольшая средняя оценка у автора книг о Гарри Поттере. Считаю, что в продукт стоит добавить книги этого автора, тем более, что они также входят в топ-10 самых оцениваемых книг

## Среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

In [16]:
# формируем запрос
query = """SELECT ROUND(AVG(SUBQ2.review_cnt), 2) AS average_number_of_reviews
           FROM (
                SELECT username  
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 50) AS SUBQ1
           LEFT JOIN (
                       SELECT username,
                              COUNT(review_id) AS review_cnt
                       FROM reviews
                       GROUP BY username) AS SUBQ2
            ON SUBQ2.username = SUBQ1.username
        """

In [17]:
# выводим результат
SQL_request_result(query)

,average_number_of_reviews
0,24.33


#### Вывод

24 обзора на пользователя - достаточно небольшое количество. Возможно стоит предусмотреть систему поощрения пользователей за обзоры. Это поможет остальным пользователям подбирать интересные для себя книги

## 3. Общие вывод

Количество книг младше 2000 года очень мало, на него не стоит ориентироваться. В топ-10 самых оцениваемых книг входит преимущественно фэнтези. Наивысшая средняя оценка у автора книги в этом жанре.

Стоит предусмотреть систему вознаграждений пользователей за оставленные отзывы о книгах. 